In [383]:
import pandas as pd
import pandera.pandas as pa

import numpy as np
import re
import sys
import os
sys.path.append(os.path.abspath(".."))
from core.viz import plot_line, create_subplot_grid, plot_bar, plot_statistical_strip
from core.s3 import S3AssetManager
from core.quality import DataValidator, create_validator


In [384]:
notebook_name = "exp_nutrinor_qa_data"
s3 = S3AssetManager(notebook_name=notebook_name)
BULTOS_TO_TON = 0.04

In [385]:
hist_pel1 = pd.read_excel("../raw/Rendimiento pelletizadoras 2024.xlsx",
 sheet_name="rendimiento pellet 1 (420)",
 skiprows=1)
hist_pel1["pellet"] = "pellet 420"

hist_pel2 = pd.read_excel("../raw/Rendimiento pelletizadoras 2024.xlsx",
 sheet_name="rendimiento pellet 2 (520)",
 skiprows=1)
hist_pel2["pellet"] = "pellet 520"

hist_pel3 = pd.read_excel("../raw/Rendimiento pelletizadoras 2024.xlsx",
 sheet_name="rendimiento pellet 3 (350)",
 skiprows=1)
hist_pel3["pellet"] = "pellet 350"

hist_pel = pd.concat([hist_pel1, hist_pel2, hist_pel3])

rename_cols = {
    "año": "year",
    "mes": "month",
    "semana": "week",
    "Fecha": "date",
    "Producto": "product",
    "Pelletizador": "pelletizer",
    "Hora inicial ": "start_time",
    "Hora final ": "end_time",
    "tiempo trabajado": "work_time",
    "tiempo trabajado (minutos)": "work_time_minutes",
    "tiempo trabajado (horas)": "work_time_hours",
    "bultos": "bags",
    "durabilidad (%)": "durability_pct",
    "carga (Hz)": "load_hz",
    "toneladas": "tons",
    "ton/hora": "tons_per_hour",
    "lote": "lot",
    "Dado": "die",
    "ROLLER": "roller",
    "Ton/Dia": "tons_per_day",
    "Horas/dia": "hours_per_day",
    "Ton/x Hora": "tons_per_hour_alt",
    "Ton/semanal": "tons_weekly",
    "Hora/semanal": "hours_weekly",
    "prom/sem": "avg_weekly",
    "Ton/mensual": "tons_monthly",
    "Hora/mensuales": "hours_monthly",
    "prom/mensual": "avg_monthly",
    "OBSERVACIONES": "notes",
}
hist_pel = hist_pel.rename(columns=rename_cols)

hist_pel["date"] = pd.to_datetime(hist_pel["date"], format="%Y-%m-%d")
hist_pel = hist_pel[hist_pel["date"].notna()]

s = hist_pel["lot"].astype(str).str.strip()
mask_digits = s.str.fullmatch(r"\d+")

hist_pel_digits = hist_pel.loc[mask_digits].copy()      
hist_pel_nondigits = hist_pel.loc[~mask_digits].copy()
hist_pel_digits['tons_cal'] = hist_pel_digits['bags'] * BULTOS_TO_TON

/Users/juandavidrincon/Documents/hawking/.venv/lib/python3.13/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/Users/juandavidrincon/Documents/hawking/.venv/lib/python3.13/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/Users/juandavidrincon/Documents/hawking/.venv/lib/python3.13/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [386]:
cls = [
'date', 'lot', 'product', 'pellet', 'start_time',
'end_time', 'work_time', 'tons_cal',
'work_time_hours', 'bags',
'durability_pct', 'load_hz', 'tons',
'tons_per_hour', 'die',
'roller', 'notes',  'pelletizer',

]

In [387]:
ref = ["date", "pellet", "lot", 'start_time']
hist_pel_digits[hist_pel_digits.duplicated(ref, keep=False)][cls].sort_values(ref)

,date,lot,product,pellet,start_time,end_time,work_time,tons_cal,work_time_hours,bags,durability_pct,load_hz,tons,tons_per_hour,die,roller,notes,pelletizer


In [388]:
cls_num = [
    'work_time',
    'tons_cal',
    'durability_pct',
    'work_time_hours',
]
for cl in cls_num:
    hist_pel_digits[cl] = pd.to_numeric(hist_pel_digits[cl], errors='coerce')

In [389]:
notes_join = lambda s: ", ".join(
    s.dropna()
     .astype(str)
     .str.strip()
     .loc[lambda x: x.ne("")]
     .unique()
)
rend_pel = hist_pel_digits.groupby(['lot']).agg(
                date=('date', 'first'),
                batch=('lot', 'count'),
                pellet=("pellet", notes_join),
                product=("product", notes_join),
                #start_time=('start_time', 'max'),
                #end_time=('end_time', 'last'),
                #work_time=('work_time', 'sum'),
                #work_time_hours=('work_time_hours', 'sum'),
                tons=('tons_cal', 'sum'),
                #durability_pct=('durability_pct', 'mean'),
                #die=('die', 'first'),
                #roller=('roller', 'first'),
                #notes=("notes", notes_join),
                #pelletizer=('pelletizer', 'first'),
).reset_index()
rend_pel["batch"].value_counts()


batch
1    5176
2    1137
3     148
4      19
5       2
6       2
Name: count, dtype: int64

In [390]:
rend_pel["pellet"].value_counts()

pellet
pellet 520                4132
pellet 420                2320
pellet 420, pellet 520      32
Name: count, dtype: int64

In [391]:
path = "../raw/SACK OFF HISTORICO.xlsx"

cols_out = ["date", "lot", "product", "theoretical_bag", "actual_bag", "difference", "sackoff", "pellet"]

def build_block(df: pd.DataFrame, cols_in: list[str], pellet_label: str) -> pd.DataFrame:
    return (
        df.loc[:, cols_in]
          .copy()
          .rename(columns={
              cols_in[0]: "date",
              cols_in[1]: "lot",
              cols_in[2]: "product",
              cols_in[3]: "theoretical_bag",
              cols_in[4]: "actual_bag",
              cols_in[5]: "difference",
              cols_in[6]: "sackoff",
          })
          .assign(pellet=pellet_label)
          .loc[:, cols_out]
    )

# Especificación de los bloques por hoja (igual a tu caso AGOSTO 25)
BLOCKS = [
    (["Fecha",   "Lote",   "Producto 420",     "Cant O.P",   "Cant.Prod",   "Diferencia",   "Sackoff"],   "pellet 420"),
    (["Fecha.1", "Lote.1", "Producto 520",     "Cant O.P.1", "Cant.Prod.1", "Diferencia.1", "Sackoff.1"], "pellet 520"),
    (["Fecha.2", "Lote.2", "Producto 350",     "Cant O.P.2", "Cant.Prod.2", "Diferencia.2", "Sackoff.2"], "pellet 350"),
    (["Fecha.3", "Lote.3", "producto harina",  "Cant O.P.3", "Cant.Prod.3", "Diferencia.3", "Sackoff.3"], "harina"),
    (["Fecha.4", "Lote.4", "producto granel",  "Cant O.P.4", "Cant.Prod.4", "Diferencia.4", "Sackoff.4"], "granel"),
]

MONTH_PAT = r"(ENERO|FEBRERO|MARZO|ABRIL|MAYO|JUNIO|JULIO|AGOSTO|SEPTIEMBRE|OCTUBRE|NOVIEMBRE|DICIEMBRE|NOV|DIC)"
re_month_sheet = re.compile(rf"^{MONTH_PAT}\s+\d{{2}}$", re.IGNORECASE)

xlf = pd.ExcelFile(path)
month_sheets = [s for s in xlf.sheet_names if re_month_sheet.match(s.strip())]

all_dfs = []
for sh in month_sheets:
    df = pd.read_excel(xlf, sheet_name=sh, skiprows=0)

    blocks = []
    for cols_in, label in BLOCKS:
        # si alguna columna no existe en esa hoja, se salta ese bloque
        if all(c in df.columns for c in cols_in):
            blocks.append(build_block(df, cols_in, label))

    if blocks:
        out = pd.concat(blocks, ignore_index=True).assign(sheet_name=sh)
        all_dfs.append(out)

hist_sackoff_by_month = pd.concat(all_dfs, ignore_index=True)
hist_sackoff_by_month["date"] = pd.to_datetime(hist_sackoff_by_month["date"], errors="coerce")
hist_sackoff_by_month = hist_sackoff_by_month[hist_sackoff_by_month["date"].notna()]

hist_sackoff_by_month["theoretical_tons"] = hist_sackoff_by_month["theoretical_bag"] * 0.04
hist_sackoff_by_month["actual_tons"] = hist_sackoff_by_month["actual_bag"] * 0.04

df = hist_sackoff_by_month.copy()

df["lot"] = (
    df["lot"]
    .astype(str)
    .str.strip()
    .str.replace(r"\.0$", "", regex=True)
    .replace({"nan": pd.NA, "None": pd.NA, "": pd.NA})
)

mask_digits = df["lot"].str.fullmatch(r"\d+").fillna(False)
hist_sackoff_month_digits = df.loc[mask_digits].copy()
hist_sackoff_month_nondigits = df.loc[~mask_digits].copy()
hist_sackoff_month_digits

,date,lot,product,theoretical_bag,actual_bag,difference,sackoff,pellet,sheet_name,theoretical_tons,actual_tons
0,2025-12-01,4861113,NUTRINOR ALTO DESEMPEÑO PELLET,800.0,805.00,5.00,0.006250,pellet 420,DICIEMBRE 25,32.00,32.20
1,2025-12-01,4861166,NUTRINOR LECHE TOP,400.0,402.00,2.00,0.005000,pellet 420,DICIEMBRE 25,16.00,16.08
2,2025-12-03,485171,NUTRINOR ALTIPLANO ALTA ENERGIA PELLET,802.0,790.00,-12.00,-0.014963,pellet 420,DICIEMBRE 25,32.08,31.60
3,2025-12-03,4911163,NUTRINOR DESEMPEÑO NORTE SOLIDOS PELLET,400.0,404.00,4.00,0.010000,pellet 420,DICIEMBRE 25,16.00,16.16
4,2025-12-03,4921161,GANADERIA PROBIOLECHE,800.0,802.00,2.00,0.002500,pellet 420,DICIEMBRE 25,32.00,32.08
...,...,...,...,...,...,...,...,...,...,...,...
11474,2024-12-30,5271128,NaN,375.0,369.25,-5.75,-0.015333,harina,DIC 24,15.00,14.77
11475,2024-12-30,527158,NaN,500.0,490.25,-9.75,-0.019500,harina,DIC 24,20.00,19.61
11476,2024-12-30,5271106,NaN,375.0,362.00,-13.00,-0.034667,harina,DIC 24,15.00,14.48
11477,2024-12-30,111129,NaN,250.0,249.75,-0.25,-0.001000,harina,DIC 24,10.00,9.99


In [392]:
hist_sackoff_month_digits[hist_sackoff_month_digits.duplicated(["lot", "pellet", "date"],keep=False)].sort_values(by="lot")

,date,lot,product,theoretical_bag,actual_bag,difference,sackoff,pellet,sheet_name,theoretical_tons,actual_tons
7441,2025-03-06,1031161,NUTRINOR DESEMPEÑO NORTE PELLET,400.0,396.0,-4.0,-0.010000,pellet 420,MARZO 25,16.00,15.84
7445,2025-03-06,1031161,GANADERIA PROBIOLECHE,800.0,782.0,-18.0,-0.022500,pellet 420,MARZO 25,32.00,31.28
7653,2025-03-08,1051140,MAQUILA CERDOS FINALIZACION HD 454,150.0,141.0,-9.0,-0.060000,pellet 520,MARZO 25,6.00,5.64
7652,2025-03-08,1051140,MAQUILA CERDOS FINALIZACION HD,100.0,104.0,4.0,0.040000,pellet 520,MARZO 25,4.00,4.16
7460,2025-03-11,111153,MAQUILA GESTACION NORMAL SUPERCERDO,150.0,147.0,-3.0,-0.020000,pellet 420,MARZO 25,6.00,5.88
...,...,...,...,...,...,...,...,...,...,...,...
8042,2025-02-28,94124,MAQUILA NUTRINOR CERDOS INICIACION MEDICADO,151.0,145.0,-6.0,-0.039735,pellet 420,FEBRERO 25,6.04,5.80
8043,2025-02-28,94124,MAQUILA NUTRINOR CERDOS INICIACION,250.0,246.0,-4.0,-0.016000,pellet 420,FEBRERO 25,10.00,9.84
8038,2025-02-28,94161,MAQUILA LEVANTE REEMPLAZO SCP,150.0,149.0,-1.0,-0.006667,pellet 420,FEBRERO 25,6.00,5.96
8039,2025-02-28,94161,MAQUILA REEMPLAZO CHOQUE SUPERCERDO,100.0,98.0,-2.0,-0.020000,pellet 420,FEBRERO 25,4.00,3.92


In [393]:
sackoff_month =  hist_sackoff_month_digits.groupby(["lot"]).agg(
    date=('date', 'first'),
    batch=('lot', 'count'),
    pellet=("pellet", notes_join),
    product=("product", notes_join),
    sheet_name=('sheet_name', notes_join),
    theoretical_tons=('theoretical_tons', 'sum'),
    actual_tons=('actual_tons', 'sum'), 
).reset_index()
sackoff_month

,lot,date,batch,pellet,product,sheet_name,theoretical_tons,actual_tons
0,1011100,2025-03-04,1,pellet 520,MAQUILA BR PRELEVANTE EB,MARZO 25,8.04,8.72
1,1011103,2025-03-04,1,pellet 520,NUTRINOR LECHE PELLET ELITE,MARZO 25,16.00,16.08
2,1011108,2025-03-04,1,harina,GANADERIA LEVANTE PLUS,MARZO 25,25.10,24.71
3,1011109,2025-03-04,1,harina,NUTRINOR ALTIPLANO EX ALTA ENER HARINA,MARZO 25,35.00,34.25
4,1011112,2025-03-04,1,pellet 420,NUTRINOR ALTA ENERGIA CP PELLET,MARZO 25,32.40,31.96
...,...,...,...,...,...,...,...,...
6252,961160,2025-03-01,1,harina,NUTRINOR ALTIPLANO EX ALTA ENER HARINA,MARZO 25,7.00,6.62
6253,961161,2025-03-02,1,harina,GANADERIA LEVANTE PLUS,MARZO 25,15.00,14.70
6254,96146,2025-03-04,1,pellet 520,MAQUILA NUTRINOR LECHONES MEDICADO,MARZO 25,32.00,31.84
6255,96171,2025-03-02,1,pellet 420,NUTRINOR ALTIPLANO EX ALTA ENER PELLET,MARZO 25,32.12,31.52


In [394]:
sackoff_month["pellet"].value_counts()

pellet
pellet 520                            1749
pellet 420                            1303
harina                                 989
granel                                 853
pellet 350                             763
pellet 350, pellet 520                  81
pellet 420, harina                      65
granel, pellet 520                      61
pellet 520, harina                      53
granel, harina                          51
pellet 520, granel                      50
pellet 350, pellet 420                  47
granel, pellet 420                      34
pellet 420, granel                      33
pellet 350, granel                      29
harina, pellet 420                      20
pellet 420, pellet 520                  14
pellet 520, pellet 350                  10
pellet 350, harina                      10
harina, granel                           7
pellet 520, pellet 420                   7
harina, pellet 520                       5
pellet 520, granel, harina               5
pell

In [395]:
sackoff_month

,lot,date,batch,pellet,product,sheet_name,theoretical_tons,actual_tons
0,1011100,2025-03-04,1,pellet 520,MAQUILA BR PRELEVANTE EB,MARZO 25,8.04,8.72
1,1011103,2025-03-04,1,pellet 520,NUTRINOR LECHE PELLET ELITE,MARZO 25,16.00,16.08
2,1011108,2025-03-04,1,harina,GANADERIA LEVANTE PLUS,MARZO 25,25.10,24.71
3,1011109,2025-03-04,1,harina,NUTRINOR ALTIPLANO EX ALTA ENER HARINA,MARZO 25,35.00,34.25
4,1011112,2025-03-04,1,pellet 420,NUTRINOR ALTA ENERGIA CP PELLET,MARZO 25,32.40,31.96
...,...,...,...,...,...,...,...,...
6252,961160,2025-03-01,1,harina,NUTRINOR ALTIPLANO EX ALTA ENER HARINA,MARZO 25,7.00,6.62
6253,961161,2025-03-02,1,harina,GANADERIA LEVANTE PLUS,MARZO 25,15.00,14.70
6254,96146,2025-03-04,1,pellet 520,MAQUILA NUTRINOR LECHONES MEDICADO,MARZO 25,32.00,31.84
6255,96171,2025-03-02,1,pellet 420,NUTRINOR ALTIPLANO EX ALTA ENER PELLET,MARZO 25,32.12,31.52


In [396]:
rev_by_month = pd.merge(
    rend_pel,
    sackoff_month,
    on=["lot"],
    how="outer",
    indicator=True,
    suffixes=("_rend_pel", "_sackoff_month"),
)

In [397]:
ord_col =[
'lot', 'date_rend_pel','date_sackoff_month', 
'batch_rend_pel', 'batch_sackoff_month',
'pellet_rend_pel', 'pellet_sackoff_month',
'product_rend_pel','product_sackoff_month',
'tons', 'theoretical_tons', 'actual_tons',
'sheet_name', '_merge'
]

In [398]:
rev_by_month["_merge"].value_counts()

_merge
left_only     3838
right_only    3611
both          2646
Name: count, dtype: int64

In [399]:
rev_by_month_both = rev_by_month[rev_by_month["_merge"] == 'both'][ord_col]

In [400]:
rev_by_month_both[rev_by_month_both["batch_rend_pel"] == rev_by_month_both["batch_sackoff_month"]]["batch_rend_pel"].value_counts()

batch_rend_pel
1.0    1650
2.0     142
3.0       9
4.0       1
Name: count, dtype: int64

In [401]:
rev_by_month_both[rev_by_month_both["batch_rend_pel"] != rev_by_month_both["batch_sackoff_month"]].shape

(844, 14)

In [402]:
rev_by_month_both[rev_by_month_both["pellet_rend_pel"] != rev_by_month_both["pellet_sackoff_month"]].shape

(1329, 14)

In [403]:
rev_by_month_both[rev_by_month_both["tons"] != rev_by_month_both["theoretical_tons"]].shape

(1290, 14)

In [404]:
rev_by_month_both[rev_by_month_both["tons"] == rev_by_month_both["theoretical_tons"]].shape

(1356, 14)

In [405]:
diferencia_tons = (rev_by_month_both[rev_by_month_both["tons"] != rev_by_month_both["theoretical_tons"]]["tons"].sum()-
 rev_by_month_both[rev_by_month_both["tons"] != rev_by_month_both["theoretical_tons"]]["theoretical_tons"].sum())
diferencia_tons

np.float64(-15155.973799999996)

In [406]:
rev_by_month_both[rev_by_month_both["tons"] != rev_by_month_both["theoretical_tons"]].sheet_name.value_counts()

sheet_name
OCTUBRE 25, AGOSTO 25               181
MARZO 25                            151
ABRIL 25                            143
SEPTIEMBRE 25                        89
JUNIO 25                             84
MAYO 25                              81
JULIO 25                             74
NOVIEMBRE 25, NOV 24                 73
OCTUBRE 24                           67
JULIO 25, JULIO 24                   47
DICIEMBRE 25, DIC 24                 40
NOVIEMBRE 25                         40
MAYO 25, MAYO 24                     36
OCTUBRE 25, AGOSTO 25, AGOSTO 24     35
SEPTIEMBRE 25, SEPTIEMBRE 24         32
OCTUBRE 25, OCTUBRE 24               20
DICIEMBRE 25                         19
OCTUBRE 25                           17
AGOSTO 25                            16
AGOSTO 25, AGOSTO 24                 13
DIC 24                               10
MAYO 24                               4
NOV 24                                3
ENERO 25                              2
AGOSTO 24                    

In [407]:
data_good = rev_by_month_both[rev_by_month_both["tons"] == rev_by_month_both["theoretical_tons"]]
data_good[data_good["pellet_rend_pel"] != data_good["pellet_sackoff_month"]]

,lot,date_rend_pel,date_sackoff_month,batch_rend_pel,batch_sackoff_month,pellet_rend_pel,pellet_sackoff_month,product_rend_pel,product_sackoff_month,tons,theoretical_tons,actual_tons,sheet_name,_merge
3569,1011109,2025-03-04,2025-03-04,1.0,1.0,pellet 520,harina,MAGRO SUPERCERDO GRANEL,NUTRINOR ALTIPLANO EX ALTA ENER HARINA,35.0,35.0,34.25,MARZO 25,both
3573,1011131,2025-03-03,2025-03-04,1.0,1.0,pellet 520,harina,NUTRIMAGRO GRANEL,NUTRINOR ALTA ENERGIA HNA CP,15.0,15.0,14.66,MARZO 25,both
3574,1011150,2025-03-03,2025-03-04,1.0,1.0,pellet 420,harina,PRODUCCION SALAS LB GRANEL,NUTRINOR ALTIPLANO EX ALTA ENER HARINA,14.0,14.0,13.89,MARZO 25,both
3575,1011160,2025-03-03,2025-03-04,1.0,1.0,pellet 520,harina,NUTRILECHON 3 GRANEL,"MAQUILA NUTRINOR MARRANAS ""G""",6.0,6.0,5.84,MARZO 25,both
3583,1021134,2025-03-04,2025-03-05,1.0,1.0,pellet 520,harina,NUTRIMAGRO MEDICADO GRANEL,NUTRINOR ALTIPLANO EX ALTA ENER HARINA,14.0,14.0,13.66,MARZO 25,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9479,5071131,2025-12-14,2025-12-15,1.0,1.0,pellet 520,granel,NUTRIMAGRO GRANEL,MAQUILA NUTRI-MAGRO GRANEL,30.0,30.0,30.04,DICIEMBRE 25,both
9488,5111108,2025-12-16,2025-12-16,1.0,1.0,pellet 520,granel,DESARROLLO SUPERCERDO GRANEL,MAQUILA DESARROLLO SUPERCERDO GRANEL,10.0,10.0,9.97,DICIEMBRE 25,both
9489,5111109,2025-12-16,2025-12-16,1.0,1.0,pellet 520,granel,MAGRO SUPERCERDO GRANEL,MAQUILA MAGRO SUPERCERDO GRANEL,33.0,33.0,32.92,DICIEMBRE 25,both
9498,5111161,2025-12-15,2025-12-16,3.0,1.0,pellet 420,granel,PROBIOLECHE GRANEL,GANADERIA PROBIOLECHE GRANEL,80.0,80.0,80.00,DICIEMBRE 25,both


In [408]:
validator = DataValidator()

In [ ]:
validator.add_check(
    pa.Check(lambda df: df["batch_rend_pel"] == df["batch_sackoff_month"], name="match_batches")
)


result = validator.validate(rev_by_month_both)

In [412]:
result.get_clean() 
result.get_errors()
result.report()

,check,unique_failed_rows
0,match_batches,844


In [ ]:
hist_sackoff_tot = pd.read_excel("../raw/SACK OFF HISTORICO.xlsx",
 sheet_name="TOTAL",
 skiprows=0)

rename_cols = {
    "AÑO": "year",
    "MES": "month",
    "FECHA": "date",
    "LOTE": "lot",
    "PRODUCTO": "product",
    "TEORICO": "theoretical_bag",
    "REAL": "actual_bag",
    "DIFERENCIA": "difference_bag",
    "SACKOFF": "sackoff",
    "ENSAQUE": "bagging",
}

hist_sackoff_tot.columns = hist_sackoff_tot.columns.map(rename_cols)
hist_sackoff_tot["date"] = pd.to_datetime(hist_sackoff_tot["date"], format="%Y-%m-%d")
hist_sackoff_tot = hist_sackoff_tot[hist_sackoff_tot["date"].notna()]


df = hist_sackoff_tot.copy()

df["lot"] = (
    df["lot"].astype(str).str.strip()
      .str.replace(r"\.0$", "", regex=True)
      .replace({"nan": pd.NA, "None": pd.NA, "": pd.NA})
)

mask_digits = df["lot"].str.fullmatch(r"\d+").fillna(False)

hist_sackoff_digits = df.loc[mask_digits].copy()
hist_sackoff_nondigits = df.loc[~mask_digits].copy()

# convertir lot a entero nullable (ya que son solo dígitos)
# (opcional) si date está como string, normaliza
hist_sackoff_digits["date"] = pd.to_datetime(hist_sackoff_digits["date"], errors="coerce")

# calcular tons (asumiendo 1 bulto = 40kg => 0.04 ton)
hist_sackoff_digits["theoretical_tons"] = hist_sackoff_digits["theoretical_bag"] * 0.04
hist_sackoff_digits["actual_tons"] = hist_sackoff_digits["actual_bag"] * 0.04
hist_sackoff_digits["difference_tons"] = hist_sackoff_digits["difference_bag"] * 0.04  # ojo nombre columna

In [300]:
hist_sackoff_digits[hist_sackoff_digits.duplicated(["lot", "date"],keep=False)].sort_values(by="lot")

,year,month,date,lot,product,theoretical_bag,actual_bag,difference_bag,sackoff,bagging,theoretical_tons,actual_tons,difference_tons
4302,2025,ENERO,2025-01-04,0151134,MAQUILA NUTRI-MAGRO GRANEL,631.0,604.50,-26.50,-0.041997,520 G,25.24,24.18,-1.06
4301,2025,ENERO,2025-01-04,0151134,MAQUILA NUTRI-MAGRO GRANEL MEDICADO,250.0,240.00,-10.00,-0.040000,520 G,10.00,9.60,-0.40
3920,2025,ENERO,2025-01-07,0161112,NUTRINOR ALTA ENERGIA CP PELLET,810.0,807.00,-3.00,-0.003704,420,32.40,32.28,-0.12
4201,2025,ENERO,2025-01-07,0161112,NUTRINOR ALTA ENERGIA HNA CP,351.0,349.00,-2.00,-0.005698,420 G,14.04,13.96,-0.08
3921,2025,ENERO,2025-01-07,021124,MAQUILA NUTRINOR CERDOS INICIACION MEDICADO,200.0,197.00,-3.00,-0.015000,420,8.00,7.88,-0.12
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4818,2025,DICIEMBRE,2024-12-26,5221129,MAQUILA NUTRI-DESARROLLO GRANEL,375.0,369.50,-5.50,-0.014667,420,15.00,14.78,-0.22
4639,2025,DICIEMBRE,2024-12-28,5251161,MAQUILA REEMPLAZOS SUPERCERDO,202.0,208.00,6.00,0.029703,420 G,8.08,8.32,0.24
4740,2025,DICIEMBRE,2024-12-28,5251161,GANADERIA PROBIOLECHE GRANEL,1350.0,1330.50,-19.50,-0.014444,520 G,54.00,53.22,-0.78
4824,2025,DICIEMBRE,2024-12-30,5271128,MAQUILA NUTRI LECHON RECIBO GRANEL,375.0,369.25,-5.75,-0.015333,420,15.00,14.77,-0.23


In [301]:
hist_sackoff = hist_sackoff_digits.groupby(["lot"]).agg(
    date=('date', 'first'),
    batch=('lot', 'count'),
    product=("product", notes_join),
    theoretical_tons=('theoretical_tons', 'sum'),
    actual_tons=('actual_tons', 'sum'), 
).reset_index()
hist_sackoff

,lot,date,batch,product,theoretical_tons,actual_tons
0,0,2024-05-06,3,CUCHUCO DE MAIZ,26.000,25.96
1,00161160,2025-01-07,1,MAQUILA NUTRI LECHON 3 GRANEL,21.000,20.37
2,011103,2025-01-02,1,NUTRINOR LECHE PELLET ELITE,16.000,15.96
3,0111129,2025-01-02,1,MAQUILA NUTRI-DESARROLLO GRANEL,27.000,25.27
4,0111131,2025-01-02,1,MAQUILA NUTRI-MAGRO GRANEL,17.000,16.10
...,...,...,...,...,...,...
6669,551129,2024-12-28,1,MAQUILA NUTRI-DESARROLLO GRANEL,15.000,14.60
6670,641131,2025-02-07,1,MAQUILA NUTRI-MAGRO GRANEL,30.000,29.45
6671,656154,2024-12-30,1,"MAQUILA MARRANAS ""L"" EB",4.000,3.84
6672,880300,2024-07-10,1,MAQUILA NUTRINOR LECHONES MEDICADO,22.080,21.36


In [302]:
hist_sackoff['batch'].value_counts()

batch
1    5667
2     891
3     103
4      13
Name: count, dtype: int64

In [307]:
rev_by_hist_sackoff = pd.merge(
    rend_pel,
    hist_sackoff,
    on=["lot"],
    how="outer",
    indicator=True,
    suffixes=("_rend_pel", "_sackoff_hist"),
)

cols_hist_rend = [
    'lot','pellet', 'date_rend_pel',  'date_sackoff_hist',
     'batch_rend_pel', 'batch_sackoff_hist',
     'product_rend_pel','product_sackoff_hist',
    'tons',  'theoretical_tons', 'actual_tons', '_merge']

rev_by_hist_sackoff["_merge"].value_counts()

_merge
right_only    4049
left_only     3859
both          2625
Name: count, dtype: int64

In [318]:
rev_by_hist_both = rev_by_hist_sackoff[rev_by_hist_sackoff["_merge"] == 'both'][cols_hist_rend]
rev_by_hist_both[rev_by_hist_both['batch_rend_pel'] != rev_by_hist_both['batch_sackoff_hist']].shape


(786, 12)

In [320]:
rev_by_hist_both[rev_by_hist_both['tons'] != rev_by_hist_both['theoretical_tons']].shape

(1334, 12)

In [324]:
# sobre la misma hoja
rev_by_same_sackoff = pd.merge(
    sackoff_month,
    hist_sackoff,
    on=["lot"],
    how="outer",
    indicator=True,
    suffixes=("_month", "_hist"),
)

In [326]:
rev_by_same_sackoff["_merge"].value_counts()

_merge
both          4948
right_only    1726
left_only     1309
Name: count, dtype: int64

In [333]:
rev_by_same_sackoff_both = rev_by_same_sackoff[rev_by_same_sackoff["_merge"] == "both"]
rev_by_same_sackoff_both[rev_by_same_sackoff_both['batch_month'] != rev_by_same_sackoff_both['batch_hist']].shape

(897, 14)

In [336]:
rev_by_same_sackoff_both[rev_by_same_sackoff_both['theoretical_tons_month'] != rev_by_same_sackoff_both['theoretical_tons_hist']].shape

(936, 14)

In [339]:
rev_by_same_sackoff_both[rev_by_same_sackoff_both['actual_tons_month'] != rev_by_same_sackoff_both['actual_tons_hist']].shape

(961, 14)